In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE244266"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE244266"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE244266.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE244266.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE244266.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association of Molecular Subtypes with Pathologic Response, PFS and OS in a Phase II Study of Coexpression Extrapolation (COXEN) with Neoadjuvant Chemotherapy (NAC) for Localized, Muscle-Invasive Bladder Cancer (SWOG S1314; NCT02177695)"
!Series_summary	"Investigation of RNA-based molecular subtypes as additional predictive biomarkers for neoadjuvant chemotherapy response, progression-free survival and survival in patients treated in S1314."
!Series_overall_design	"randomized 2-arm phase II trial of neoadjuvant chemotherapy in muscle-invasive bladder cancer, translational analysis of tissue"
Sample Characteristics Dictionary:
{0: ['treatment arm: DDMVAC+CYST', 'treatment arm: GC+CYST'], 1: ['disease: muscle-invasive bladder cancer'], 2: ['clinical_stage_strat_factor: Clinical Stage -T2', 'clinical_stage_strat_factor: Clinical Stage -T3 or T4a']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Determine gene expression data availability
# Based on the information provided, this dataset appears to be a clinical trial (S1314) for bladder cancer
# with molecular subtypes (RNA-based) analysis, which suggests gene expression data.
is_gene_available = True

# Step 2: Identify keys and create conversion functions

# 2.1: Trait (Bladder Cancer)
# From sample characteristics, index 1 shows "disease: muscle-invasive bladder cancer"
trait_row = 1

def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since this is a bladder cancer study, convert to binary
    if 'bladder cancer' in value.lower() or 'muscle-invasive bladder cancer' in value.lower():
        return 1
    return 0

# 2.2: Age
# There's no age information in the sample characteristics
age_row = None

def convert_age(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3: Gender
# There's no gender information in the sample characteristics
gender_row = None

def convert_gender(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# Step 3: Save metadata - perform initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Extract clinical features if available
if trait_row is not None:
    # Create clinical dataframe using the geo_select_clinical_features function
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Preview of selected clinical features:")
    print(preview_df(clinical_selected))
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    clinical_selected.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM7810144': [1.0], 'GSM7810145': [1.0], 'GSM7810146': [1.0], 'GSM7810147': [1.0], 'GSM7810148': [1.0], 'GSM7810149': [1.0], 'GSM7810150': [1.0], 'GSM7810151': [1.0], 'GSM7810152': [1.0], 'GSM7810153': [1.0], 'GSM7810154': [1.0], 'GSM7810155': [1.0], 'GSM7810156': [1.0], 'GSM7810157': [1.0], 'GSM7810158': [1.0], 'GSM7810159': [1.0], 'GSM7810160': [1.0], 'GSM7810161': [1.0], 'GSM7810162': [1.0], 'GSM7810163': [1.0], 'GSM7810164': [1.0], 'GSM7810165': [1.0], 'GSM7810166': [1.0], 'GSM7810167': [1.0], 'GSM7810168': [1.0], 'GSM7810169': [1.0], 'GSM7810170': [1.0], 'GSM7810171': [1.0], 'GSM7810172': [1.0], 'GSM7810173': [1.0], 'GSM7810174': [1.0], 'GSM7810175': [1.0], 'GSM7810176': [1.0], 'GSM7810177': [1.0], 'GSM7810178': [1.0], 'GSM7810179': [1.0], 'GSM7810180': [1.0], 'GSM7810181': [1.0], 'GSM7810182': [1.0], 'GSM7810183': [1.0], 'GSM7810184': [1.0], 'GSM7810185': [1.0], 'GSM7810186': [1.0], 'GSM7810187': [1.0], 'GSM7810188': [1.0], 'GSM7810189': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers appear to be Affymetrix probe IDs from a microarray platform,
# not standard human gene symbols. For example, "1007_s_at" follows the Affymetrix naming pattern.
# These will need to be mapped to official gene symbols for standardized analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying columns for gene mapping
# Looking at gene annotation, we need to map from the 'ID' column (Affymetrix probe IDs) to 'Gene Symbol' column
print("Columns available in gene_annotation:")
print(gene_annotation.columns)

# 2. Get a gene mapping dataframe
# Using get_gene_mapping function from the library to extract these columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# Using apply_gene_mapping function from the library
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])


Columns available in gene_annotation:
Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')


Gene mapping dataframe shape: (45782, 2)
First few rows of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression dataframe shape after mapping: (21278, 189)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data from the matrix file again to ensure we have the correct sample IDs
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

print("Clinical data structure:")
print(clinical_data.shape)
print("First few rows of clinical data:")
print(clinical_data.head())

# Extract clinical features with the correct sample IDs
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape after extraction: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Save the clinical data for inspection
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to match sample IDs between clinical and gene expression data."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = "Dataset contains gene expression data from bladder cancer samples with molecular subtyping information."
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=linked_data, 
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 21278
Normalized gene count: 19845


Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE244266.csv


Clinical data structure:
(3, 190)
First few rows of clinical data:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   

                                        GSM7810144  \
0                       treatment arm: DDMVAC+CYST   
1          disease: muscle-invasive bladder cancer   
2  clinical_stage_strat_factor: Clinical Stage -T2   

                                          GSM7810145  \
0                         treatment arm: DDMVAC+CYST   
1            disease: muscle-invasive bladder cancer   
2  clinical_stage_strat_factor: Clinical Stage -T...   

                                        GSM7810146  \
0                       treatment arm: DDMVAC+CYST   
1          disease: muscle-invasive bladder cancer   
2  clinical_stage_strat_factor: Clinical Stage -T2   

                                        GSM7810147  \
0                           treatment arm: GC+CYST   
1          disease: muscle-in

Linked data shape after handling missing values: (189, 19846)
Quartiles for 'Bladder_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bladder_Cancer' in this dataset is severely biased.

The dataset was determined to be not usable for analysis due to bias in the trait distribution.
